In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import StaleElementReferenceException

import urllib.robotparser
import pandas as pd
import time
import datetime

debug = True

In [2]:
############### User Agent ###############
def get_user_agent():
    
    '''
    returns:
    Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) 
    AppleWebKit/537.36 (KHTML, like Gecko) 
    Chrome/77.0.3865.90 Safari/537.36'
    '''
    driver = webdriver.Chrome()

    #Store agent in a variable and print the value
    agent = driver.execute_script("return navigator.userAgent;")
    return agent

agent = get_user_agent()

In [3]:
############### Webdriver ###############
opts = Options()
opts.add_argument(agent)
browser = webdriver.Chrome(executable_path = 'chromedriver', chrome_options = opts)
browser.implicitly_wait(10)

/Users/biancaorozco/anaconda3/envs/metis/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: use options instead of chrome_options
  after removing the cwd from sys.path.


In [4]:
############### Travel Services ###############
# Options
flights = "//button[@id='tab-flight-tab-hp']"
hotels = "//button[@id='tab-hotel-tab-hp']"
cars = "//button[@id='tab-car-tab-hp']"
vacay_rentals = "//button[@id='tab-vacation-rental-tab-hp']"

# Choose Travel Service
def choose_service(service):
    
    service_type = browser.find_element_by_xpath(service)
    
    service_type.click()

In [5]:
############### Flights ###############
# Options
roundtrip = "//label[@id='flight-type-roundtrip-label-hp-flight']"
one_way = "//label[@id='flight-type-one-way-label-hp-flight']"
multi_city = "//label[@id='flight-type-multi-dest-label-hp-flight']"

# Choose Flight Option
def choose_flight(flight):
    
    flight_type = browser.find_element_by_xpath(flight)
    
    flight_type.click()

In [6]:
############### Cities ###############
list_of_cities = ["Atlanta ATL", "Los Angeles LAX", "Chicago ORD", "Dallas DFW", "Denver DEN", "New York JFK", "San Francisco SFO", "Seattle SEA"]#, "Las Vegas LAS", "Orlando MCO", "Charlotte CLT", "Phoenix PHX", "Houston IAH", "Miami MIA", "Boston BOS", "Minneapolis MSP", "Fort Lauderdale FLL", "Detroit DTW", "Philadelphia PHL", "Baltimore BWI", "Salt Lake City SLC", "San Diego SAN", "Washington, D.C. IAD", "Tampa TPA", "Portland PDX", "Honolulu HNL"]

# Shuffle Cities List
def shuffle_cities(list_of_cities):
    list_len = len(list_of_cities)
    print(list_len)
    assert list_len % 2 == 0
    assert len(list_of_cities) == len(set(list_of_cities))
    combined_lists = []

    first_half = list_of_cities[:list_len//2]
    second_half = list_of_cities[list_len//2:]

    for i in range(list_len - 2):
        last_item_first_half = first_half.pop(-1)
        first_item_second_half = second_half.pop(0)

        first_half.insert(1, first_item_second_half)
        second_half.append(last_item_first_half)

        combined_lists.append(list(zip(first_half, second_half)))
    return combined_lists

# Execute
city_pairs = shuffle_cities(list_of_cities)

8


In [7]:
############### Departure City ###############
def select_departure_city(departure_city):
    
    try:
        fly_from = browser.find_element_by_xpath("//input[@id='flight-origin-hp-flight']")
        fly_from.clear()
        fly_from.send_keys(' ' + departure_city)
        
    except StaleElementReferenceException:
        pass
    
    # Selects first suggestion from auto-complete drop-down menu
    try:
        first_item = browser.find_element_by_xpath("//a[@id='aria-option-0']")
        first_item.click()
        
    except StaleElementReferenceException:
        pass

In [8]:
############### Arrival City ###############
def select_arrival_city(arrival_city):
    try:
        fly_to = browser.find_element_by_xpath("//input[@id='flight-destination-hp-flight']")
        fly_to.clear()
        fly_to.send_keys(' ' + arrival_city)
    except StaleElementReferenceException:
        pass

    
    # Selects first suggestion from auto-complete drop-down menu
    try:
        first_item = browser.find_element_by_xpath("//a[@id='aria-option-0']")
        first_item.click()
    except StaleElementReferenceException:
        pass

In [9]:
############### Date ###############
day = '10'
month = '10'
year = '2019'

# Input Departure Date
def choose_departure_date(day, month, year):
    dept_date_button = browser.find_element_by_xpath("//input[@id='flight-departing-single-hp-flight']")
    dept_date_button.clear()
    dept_date_button.send_keys(month + '/' + day + '/' + year)

In [10]:
############### Click Search ###############
def select_search():
    search = browser.find_element_by_xpath("//button[@class='btn-primary btn-action gcw-submit']")
    search.submit()

In [11]:
############### Data Frame ###############
df = pd.DataFrame()
def combine_data(day, month, year, from_city, to_city):
    global df
    global departure_times_list
    global arrival_times_list
    global airlines_list
    global durations_list
    global stops_list
    global airport_info_list
    global prices_list

    # adding new rows after existing ones
    csv_index = len(df.index)

    # Departure Times
    try:
        departure_times = browser.find_elements_by_xpath("//span[@data-test-id='departure-time']")
        departure_times_list = [value.text for value in departure_times]
    except StaleElementReferenceException:
        pass

    # Arrival Times
    try:
        arrival_times = browser.find_elements_by_xpath("//span[@data-test-id='arrival-time']")
        arrival_times_list = [value.text for value in arrival_times]
    except StaleElementReferenceException:
        pass

    # Airline
    try:
        airlines = browser.find_elements_by_xpath("//span[@data-test-id='airline-name']")
        airlines_list = [value.text for value in airlines]
    except StaleElementReferenceException:
        pass

    # Duration
    try:
        durations = browser.find_elements_by_xpath("//span[@data-test-id='duration']")
        durations_list = [value.text for value in durations]
    except StaleElementReferenceException:
        pass

    # Stops
    try:
        stops = browser.find_elements_by_xpath("//span[@class='number-stops']")
        stops_list = [value.text for value in stops]
    except StaleElementReferenceException:
        pass

    # Price
    try:
        prices = browser.find_elements_by_xpath("//span[@data-test-id='listing-price-dollars']")
        prices_list = [value.text for value in prices]
    except StaleElementReferenceException:
        pass


# I understand this code!!!!
    for i in range(len(departure_times_list)): # This is just for the sake of getting length of rows
        # First column: Numbering flight data
        try:
            df.loc[int(i) + csv_index, 'id_flight'] = int(i)
        except Exception as e:
            pass
        # Second column: Departure Times
        try:
            df.loc[i + csv_index, 'departure_time'] = departure_times_list[i]
        except Exception as e:
            pass
        # Third column: Arrival Times
        try:
            df.loc[i + csv_index, 'arrival_time'] = arrival_times_list[i]
        except Exception as e:
            pass
        # Fourth column: Airline
        try:
            df.loc[i + csv_index, 'airline'] = airlines_list[i]
        except Exception as e:
            pass
        # Fifth column: Flight Duration
        try:
            df.loc[i + csv_index, 'duration'] = durations_list[i]
        except Exception as e:
            pass
        # Sixth column: Number of Stops
        try:
            df.loc[i + csv_index, 'number_stops'] = stops_list[i]
        except Exception as e:
            pass
        # Seventh column: Departure Airport
        try:
            df.loc[i + csv_index, 'departure_airport'] = from_city
        except Exception as e:
            pass
        # Eigth column: Arrival Airport
        try:
            df.loc[i + csv_index, 'arrival_airport'] = to_city
        except Exception as e:
            pass
        # Ninth column: Prices
        try:
            df.loc[i + csv_index, 'prices'] = prices_list[i]
        except Exception as e:
            pass    
                # Ninth column: Prices
        try:
            df.loc[i + csv_index, 'date'] = month + '/' + day + '/' + year
        except Exception as e:
            pass   

In [12]:
############### Execute ###############
parser = urllib.robotparser.RobotFileParser()
parser.set_url("https://www.expedia.com/robots.txt")
parser.read()

if parser.can_fetch("*", "https://www.expedia.com/Flights"):

    for i in range(0,len(city_pairs)): # length = 24 lists
        for j in range(0,len(city_pairs[0])): # length = 13 pairs in each list
            link = 'https://www.expedia.com/'
            browser.get(link)
            choose_service(flights)
            choose_flight(one_way)
            from_city = ''
            to_city = ''
            for k in (0,len(city_pairs[0][0])): # length = 2 cities in each pair
            
                
                if k == 0:
                    from_city = city_pairs[i][j][k]
                    select_departure_city(from_city)
                    
                else:
                    to_city = city_pairs[i][j][1]
                    select_arrival_city(to_city)
                    choose_departure_date(day, month, year)
                    select_search()

            combine_data(day, month, year, from_city, to_city)

            if (debug):
                print(df)
                
    df.to_csv('expedia_' + month + '_'+ day + '_' + year +'.csv', header=True, index=False)
else:
    print('Error: No permission to extract this data')

   id_flight departure_time arrival_time airline duration number_stops  \
0        0.0        12:20pm       2:39pm   Delta   2h 19m    (Nonstop)   
1        1.0         5:48pm       8:15pm   Delta   2h 27m    (Nonstop)   
2        2.0         9:25pm      11:39pm   Delta   2h 14m    (Nonstop)   
3        3.0         2:03pm       4:20pm   Delta   2h 17m    (Nonstop)   
4        4.0        10:50am       1:00pm   Delta   2h 10m    (Nonstop)   
5        5.0         7:09pm       9:30pm   Delta   2h 21m    (Nonstop)   
6        6.0         3:09pm       5:35pm   Delta   2h 26m    (Nonstop)   
7        7.0         8:26am      10:38am   Delta   2h 12m    (Nonstop)   

  departure_airport arrival_airport prices        date  
0       Atlanta ATL    New York JFK   $304  10/10/2019  
1       Atlanta ATL    New York JFK   $304  10/10/2019  
2       Atlanta ATL    New York JFK   $359  10/10/2019  
3       Atlanta ATL    New York JFK   $359  10/10/2019  
4       Atlanta ATL    New York JFK   $444  10/1

     id_flight departure_time arrival_time            airline duration  \
0          0.0        12:20pm       2:39pm              Delta   2h 19m   
1          1.0         5:48pm       8:15pm              Delta   2h 27m   
2          2.0         9:25pm      11:39pm              Delta   2h 14m   
3          3.0         2:03pm       4:20pm              Delta   2h 17m   
4          4.0        10:50am       1:00pm              Delta   2h 10m   
..         ...            ...          ...                ...      ...   
107       30.0        12:06pm       4:29pm  American Airlines   4h 23m   
108       31.0         1:30pm       5:55pm  American Airlines   4h 25m   
109       32.0        10:10am       2:37pm  American Airlines   4h 27m   
110       33.0         3:07pm       7:42pm  American Airlines   4h 35m   
111       34.0         6:55am      11:31am  American Airlines   4h 36m   

    number_stops departure_airport arrival_airport prices        date  
0      (Nonstop)       Atlanta ATL    N

     id_flight departure_time arrival_time            airline duration  \
0          0.0        12:20pm       2:39pm              Delta   2h 19m   
1          1.0         5:48pm       8:15pm              Delta   2h 27m   
2          2.0         9:25pm      11:39pm              Delta   2h 14m   
3          3.0         2:03pm       4:20pm              Delta   2h 17m   
4          4.0        10:50am       1:00pm              Delta   2h 10m   
..         ...            ...          ...                ...      ...   
309       29.0         1:59pm       9:35pm  Multiple Airlines  10h 36m   
310       30.0         5:20am      10:15am  American Airlines   7h 55m   
311       31.0         1:54pm       7:38pm  American Airlines   8h 44m   
312       32.0         3:25pm       9:53pm  American Airlines   9h 28m   
313       33.0        12:23pm       7:38pm  American Airlines  10h 15m   

    number_stops departure_airport arrival_airport prices        date  
0      (Nonstop)       Atlanta ATL    N

     id_flight departure_time arrival_time airline duration number_stops  \
0          0.0        12:20pm       2:39pm   Delta   2h 19m    (Nonstop)   
1          1.0         5:48pm       8:15pm   Delta   2h 27m    (Nonstop)   
2          2.0         9:25pm      11:39pm   Delta   2h 14m    (Nonstop)   
3          3.0         2:03pm       4:20pm   Delta   2h 17m    (Nonstop)   
4          4.0        10:50am       1:00pm   Delta   2h 10m    (Nonstop)   
..         ...            ...          ...     ...      ...          ...   
479       29.0        11:29am      11:39pm  United  10h 10m     (1 stop)   
480       30.0         6:00am       6:15pm  United  10h 15m     (1 stop)   
481       31.0         6:15am       6:33pm  United  10h 18m     (1 stop)   
482       32.0         7:15am       7:43pm  United  10h 28m     (1 stop)   
483       33.0         6:00am       2:07pm   Delta    6h 7m     (1 stop)   

    departure_airport arrival_airport prices        date  
0         Atlanta ATL    New

     id_flight departure_time arrival_time airline duration number_stops  \
0          0.0        12:20pm       2:39pm   Delta   2h 19m    (Nonstop)   
1          1.0         5:48pm       8:15pm   Delta   2h 27m    (Nonstop)   
2          2.0         9:25pm      11:39pm   Delta   2h 14m    (Nonstop)   
3          3.0         2:03pm       4:20pm   Delta   2h 17m    (Nonstop)   
4          4.0        10:50am       1:00pm   Delta   2h 10m    (Nonstop)   
..         ...            ...          ...     ...      ...          ...   
697       30.0         9:50am       7:39pm  United   8h 49m     (1 stop)   
698       31.0         6:50pm      11:56pm   Delta    4h 6m     (1 stop)   
699       32.0        11:10am       4:30pm   Delta   4h 20m     (1 stop)   
700       33.0         6:00am      11:27am   Delta   4h 27m     (1 stop)   
701       34.0         7:00am       1:00pm   Delta    5h 0m     (1 stop)   

    departure_airport arrival_airport prices        date  
0         Atlanta ATL    New

     id_flight departure_time arrival_time          airline duration  \
0          0.0        12:20pm       2:39pm            Delta   2h 19m   
1          1.0         5:48pm       8:15pm            Delta   2h 27m   
2          2.0         9:25pm      11:39pm            Delta   2h 14m   
3          3.0         2:03pm       4:20pm            Delta   2h 17m   
4          4.0        10:50am       1:00pm            Delta   2h 10m   
..         ...            ...          ...              ...      ...   
918       22.0        11:11am       1:28pm            Delta      NaN   
919       23.0         7:25am       9:44am            Delta      NaN   
920       24.0         9:00am      11:21am            Delta      NaN   
921       25.0         5:30pm       7:40pm  Alaska Airlines      NaN   
922       26.0         6:24pm       8:36pm           United      NaN   

    number_stops departure_airport    arrival_airport prices        date  
0      (Nonstop)       Atlanta ATL       New York JFK   $304

In [49]:
df3 = pd.read_csv("/Users/biancaorozco/Desktop/Metis/week02/project-02/airfare_project2/expedia_10_10_2019.csv")
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 923 entries, 0 to 922
Data columns (total 10 columns):
id_flight            923 non-null float64
departure_time       923 non-null object
arrival_time         923 non-null object
airline              923 non-null object
duration             898 non-null object
number_stops         896 non-null object
departure_airport    923 non-null object
arrival_airport      923 non-null object
prices               891 non-null object
date                 923 non-null object
dtypes: float64(1), object(9)
memory usage: 72.2+ KB


In [45]:
nan_check = pd.DataFrame.isna(df3)
nan_check['duration'].count()

923

In [53]:
df3.isna().sum()

id_flight             0
departure_time        0
arrival_time          0
airline               0
duration             25
number_stops         27
departure_airport     0
arrival_airport       0
prices               32
date                  0
dtype: int64

In [54]:
clean_df = df3.dropna()

In [55]:
clean_df.isna().sum()

id_flight            0
departure_time       0
arrival_time         0
airline              0
duration             0
number_stops         0
departure_airport    0
arrival_airport      0
prices               0
date                 0
dtype: int64